In [1]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
from bs4 import BeautifulSoup
import requests

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_friendship(self, person1_name, person2_name,total,Salud,Ataque,Defensa,
                          Velocidad_atck,Velocidad_defen,Velocidad):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._create_and_return_friendship, person1_name, person2_name,total,Salud,
                Ataque,Defensa,Velocidad_atck,Velocidad_defen,Velocidad)
            for row in result:
                print("El pokemon {p1} es tipo {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod #Importacion de los datos
    def _create_and_return_friendship(tx, person1_name, person2_name, person3_name, person4_name, 
                                      person5_name, person6_name, person7_name, person8_name, 
                                      person9_name):
        
        query = ( 
            "CREATE (p1:Pokemon { name: $person1_name }) "
            "CREATE (p2:Pokemon { name: $person2_name }) "
            "CREATE (p3:Pokemon { name: $person3_name }) "
            "CREATE (p4:Pokemon { name: $person4_name }) "
            "CREATE (p5:Pokemon { name: $person5_name }) "
            "CREATE (p6:Pokemon { name: $person6_name }) "
            "CREATE (p7:Pokemon { name: $person7_name }) "
            "CREATE (p8:Pokemon { name: $person8_name }) "
            "CREATE (p9:Pokemon { name: $person9_name }) "
            "CREATE (p1)-[:TIPO]->(p2) "
            "CREATE (p1)-[:SALUD]->(p4) "
            "CREATE (p1)-[:ATAQUE]->(p5) "
            "CREATE (p1)-[:DEFENSA]->(p6) "
            "CREATE (p1)-[:VELOCIDAD_ATAQUE]->(p7) "
            "CREATE (p1)-[:VELOCIDAD_DEFENSA]->(p8) "
            "CREATE (p1)-[:VELOCIDAD]->(p9) "
            "CREATE (p1)-[:TOTAL]->(p3) "

            "RETURN p1, p2,p3,p4,p5,p6,p7,p8,p9"
        )
        result = tx.run(query, person1_name=person1_name, person2_name=person2_name, person3_name=person3_name, 
                        person4_name=person4_name, person5_name=person5_name, person6_name=person6_name, 
                        person7_name=person7_name, person8_name=person8_name, person9_name=person9_name)
        try:
            return [{"p1": row["p1"]["name"], "p2": row["p2"]["name"],"p3": row["p3"]["name"], 
                     "p4": row["p4"]["name"],"p5": row["p5"]["name"], "p6": row["p6"]["name"],
                     "p7": row["p7"]["name"], "p8": row["p8"]["name"], "p9": row["p9"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
            

##############################################
            #Credenciales#
uri = "neo4j+s://82800cbf.databases.neo4j.io"
user = "neo4j"
password = "_xxWNLqX4k3ZRq3nfj3wfL2uiWEC8Q2Lb7opgA78MnQ"
app = App(uri, user, password)
###############################################

###############################################
            #Web Scraping#
tablelist=[]
url='https://pokemondb.net/pokedex/all'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
league = soup.find('table',class_ = 'data-table block-wide')
for team in league.find_all('tbody'):
    rows = team.find_all('tr')
    for row in rows: 
        nombre = row.find('td', class_ = 'cell-name').text.strip()
        clase = row.find('td', class_ = 'cell-icon').text.strip()
        total = row.find('td', class_ = 'cell-total').text.strip()
        Salud = row.find_all('td',class_='cell-num')[1].text
        Ataque = row.find_all('td',class_='cell-num')[2].text
        Defensa = row.find_all('td',class_='cell-num')[3].text
        Velocidad_atck = row.find_all('td',class_='cell-num')[4].text
        Velocidad_defen = row.find_all('td',class_='cell-num')[5].text
        Velocidad = row.find_all('td',class_='cell-num')[6].text
        teaminLeague = {
            'Nombre': nombre,
            'Tipo': clase,
            'Total': total,
            'Salud ': Salud,
            'Ataque':Ataque,
            'Defensa':Defensa,
            'Velocidad de Ataque': Velocidad_atck,
            'Velocidad de defensa': Velocidad_defen,
            'Velocidad': Velocidad
         }
        tablelist.append(teaminLeague)


        if __name__ == "__main__": ###########Enviar los datos 
            # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
            app.create_friendship(nombre, clase,total,Salud,Ataque,Defensa,Velocidad_atck,Velocidad_defen,Velocidad)
            app.close()
            
        



ValueError: Cannot resolve address 82800cbf.databases.neo4j.io:7687

In [2]:
import pandas as pd
#################Creacion de un CSV
df= pd.DataFrame(tablelist)
df.to_csv('Tema Libre.csv')



In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

cliente = MongoClient('localhost',27017)

db = cliente["Scraping"]
listas = db.listas

url='https://pokemondb.net/pokedex/all'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
league = soup.find('table',class_ = 'data-table block-wide')
for team in league.find_all('tbody'):
    rows = team.find_all('tr')
    for row in rows: 
        num = row.find('td', class_ = 'cell-num cell-fixed').text.strip()
        nombre = row.find('td', class_ = 'cell-name').text.strip()
        clase = row.find('td', class_ = 'cell-icon').text.strip()
        total = row.find('td', class_ = 'cell-total').text.strip()
        Salud = row.find_all('td',class_='cell-num')[1].text
        Ataque = row.find_all('td',class_='cell-num')[2].text
        Defensa = row.find_all('td',class_='cell-num')[3].text
        Velocidad_atck = row.find_all('td',class_='cell-num')[4].text
        Velocidad_defen = row.find_all('td',class_='cell-num')[5].text
        Velocidad = row.find_all('td',class_='cell-num')[6].text
        
        teaminLeague = [{
            '#': num,
            'Nombre': nombre,
            'Tipo': clase,
            'Total': total,
            'Salud ': Salud,
            'Ataque':Ataque,
            'Defensa':Defensa,
            'Velocidad de Ataque': Velocidad_atck,
            'Velocidad de defensa': Velocidad_defen,
            'Velocidad': Velocidad
         }]
        listas.insert_many(teaminLeague)

